# Langevin Simulations

In [1]:
using DifferentialEquations

In [2]:
using Plots
plotly()

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots /Users/nathanielmonpere/.julia/packages/Plots/rNwM4/src/backends.jl:363


Plots.PlotlyBackend()

In [3]:
function brownianLangevin(n::Function, E, σ, u0, tspan)

    l(t) = 1/(√2*σ*n(t))

    γ(t) = √(E)/l(t) * √(π/2)
    D(t) = E*γ(t)

    function drift(du, u, p, t)
        du[1] = - γ(t)*u[1]
        du[2] = - γ(t)*u[2]
    end

    function diff(du, u, p, t)
        du[1] = √(2*D(t))
        du[2] = √(2*D(t))
    end
    
    return SDEProblem(drift, diff, u0, tspan)
end

function logisticGrowth(t, ρ, k, n0)
    return k/( 1 + (k-n0)/n0 * exp(-ρ*t) )
end

logisticGrowth (generic function with 1 method)

In [4]:
n0 = 50

ρ = 0.06
k = 400
volume = 10. * 10.
radius = 0.08
σc = 2*2radius
s0 = 0.05
E = s0^2/2
nDensity(t) = logisticGrowth(t, ρ, k, n0) / volume

u0 = [s0*cos(π), s0*sin(π)]
evolveTime = 100.
tspan = (0., evolveTime)

langevinProb = brownianLangevin(nDensity, E, σc, u0, tspan);
lSol = solve(langevinProb, SRIW1());

In [5]:
plot(lSol, vars=(1,2))
xlabel!("x velocity")
ylabel!("y velocity")

<!DOCTYPE html>
 
 
 Plots.jl

In [6]:
function velToPosition(vSol::RODESolution, pos0, times)
    pos_t_xy = Array{Float64, 2}(undef, length(times), 2)
    pos_t_xy[1, :] = pos0
    for (i, t) in enumerate(times[2:end])
        pos_t_xy[1+i, :] = pos_t_xy[i, :] .+ vSol(t) .*(times[i+1]-times[i])
    end
    return pos_t_xy
end

velToPosition (generic function with 1 method)

In [7]:
solPos = velToPosition(lSol, [0., 0.], range(0,100; length=100))
p1 = plot(solPos[:,1], solPos[:,2])
xlabel!("x position")
ylabel!("y position")
display(p1)

<!DOCTYPE html>
 
 
 Plots.jl

## Mean Squared Displacement

In [23]:
using LinearAlgebra
using Statistics
using Distributions

In [9]:
function velToPosition(ensSol::EnsembleSolution, p0, times)
    pos_Traj_t_xy = Array{Array{Float64, 2}, 1}(undef, length(ensSol))
    for (i, vTraj) in enumerate(ensSol)
        pos_Traj_t_xy[i] = velToPosition(vTraj, p0, times)
    end
    return pos_Traj_t_xy
end

function msd(ensSol, tspan::Tuple{Real, Real}, steps::Int=100)
    
    times_t = range(tspan[1], tspan[2]; length=steps)
    
    p0 = [0., 0.]
    pos_Traj_t_xy = velToPosition(ensSol, p0, times_t)
    msd_t = Array{Float64, 1}(undef, steps)

    for (i,t) in enumerate(times_t)
        msd_t[i] = mean([norm(pos_t_xy[i,:]-pos_t_xy[1,:])^2 for pos_t_xy in pos_Traj_t_xy])
    end 
    
    return times_t, msd_t
end 

msd (generic function with 2 methods)

### No growth, varying densities

In [29]:
function runWithDensity(;n0=100, volume=100, radius=0.08, sAv=0.05, evolveTime=300., runs=500)
    σc = 2*2radius
    E = sAv^2 * 2/π
    nDensity(t) = n0 / volume

    u0 = [sAv*cos(π), sAv*sin(π)]
    tspan = (0., evolveTime)
    langProb = brownianLangevin(nDensity, E, σc, u0, tspan);
    
    function randSpeedProblem(prob,i,repeat)
        s = rand(  Rayleigh( norm(prob.u0)*√(2/π) )  )
        α = rand()*2π
        @. prob.u0 = [s*cos(α), s*sin(α)]
        return prob
    end

    ensembleprob = EnsembleProblem(langProb; prob_func=randSpeedProblem)
    ensSol = solve(ensembleprob, SRIW1(), EnsembleThreads(); trajectories=runs)
    return ensSol
end

runWithDensity (generic function with 1 method)

In [30]:
ensSol = runWithDensity(n0=50, evolveTime=300., runs=1000)
times_t, msd_t = msd(ensSol, (0, 300), 501)
p1 = plot(times_t, msd_t, label="n=50")
for n in [100, 200, 300, 400]
    ensSol = runWithDensity(n0=n, evolveTime=300., runs=1000)
    times_t, msd_t = msd(ensSol, (0, 300), 201)
    plot!(times_t, msd_t, label="n="*string(n))
end
display(p1)

<!DOCTYPE html>
 
 
 Plots.jl

### Growth

In [31]:
function runWithGrowth(;n0=50, ρ=0.06, k=400, volume=100, radius=0.08, s0=0.05, evolveTime=300., runs=500)
    σc = 2*2radius
    E = s0^2 /2
    nDensity(t) = logisticGrowth(t, ρ, k, n0) / volume

    u0 = [s0*cos(π), s0*sin(π)]
    tspan = (0., evolveTime)
    langProb = brownianLangevin(nDensity, E, σc, u0, tspan);

    function randSpeedProblem(prob,i,repeat)
        s = rand(  Rayleigh( norm(prob.u0)*√(2/π) )  )
        α = rand()*2π
        @. prob.u0 = [s*cos(α), s*sin(α)]
        return prob
    end

    ensembleprob = EnsembleProblem(langProb; prob_func=randSpeedProblem)
    ensSol = solve(ensembleprob, SRIW1(), EnsembleThreads(); trajectories=runs)
    return ensSol
end

runWithGrowth (generic function with 1 method)

In [32]:
ensSol = runWithGrowth(runs=1000)
times_t, msd_t = msd(ensSol, (0, 200), 501);

In [33]:
plot(times_t, msd_t)

<!DOCTYPE html>
 
 
 Plots.jl

## Velocity Correlations

In [15]:
function velCorrelation(ensVSol, tspan::Tuple{Real, Real}, steps::Int=100)
    times_t = range(tspan[1], tspan[2]; length=steps)
    vCorr_t = Array{Float64, 1}(undef, steps)
    for (i,t) in enumerate(times_t)
        vCorr_t[i] = mean( [ vSol(t)⋅vSol(tspan[1]) for vSol in ensVSol ] )
    end
    return times_t, vCorr_t
end

velCorrelationTheory(τ, E, γ) = 2E * exp(-γ*τ)

function velCorrelationTheory(τ; N, volume, radius, s0)
    σc = 2*2radius
    E = s0^2 /2
    n = N/volume
    l = 1/(√2*σc*n)
    γ = √(E)/l * √(π/2)
    return velCorrelationTheory(τ, E, γ)
end


velCorrelationTheory (generic function with 2 methods)

In [21]:
ensSol = runWithDensity(n0=100, evolveTime=500, runs=1000)
times_t, vCorr_t = velCorrelation(ensSol, (200,400));

In [22]:
p1 = plot(times_t, vCorr_t, label="simulation")
plot!(times_t, velCorrelationTheory.(times_t.-times_t[1]; N=100, volume=100., radius=0.08, s0=0.05), label="theory")
display(p1)

<!DOCTYPE html>
 
 
 Plots.jl